In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision.datasets import CIFAR10
import pytorch_lightning as pl

In [23]:
def seed_everything(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

class CReLU(nn.Module):
    def __init__(self):
        super(CReLU, self).__init__()
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.relu
        out = torch.cat((out(x), out(-x)), dim=1)
        return out

class PGLU(nn.Module):
    def __init__(self, channels_in, channels_out, kernel_size=1, padding=0):
        super(PGLU, self).__init__()
        self.conv = nn.Conv2d(channels_in, channels_out, kernel_size, padding=padding)
        self.relu = nn.ReLU()

    def forward(self, gate, output):
        gate = self.conv(gate)
        gate = self.relu(gate)
        gate = gate > 0
        return output * gate

class OGLU(nn.Module):
    def __init__(self, channels, kernel_size=1, padding=0):
        super(OGLU, self).__init__()
        self.conv = nn.Conv2d(channels, channels, kernel_size, padding=padding)
        self.relu = nn.ReLU()

    def forward(self, output):
        gate = self.conv(output)
        gate = self.relu(gate)
        gate = gate > 0
        return output * gate

class APGLU(nn.Module):
    def __init__(self, channels_in, channels_out, kernel_size=1, padding=0):
        super(APGLU, self).__init__()
        self.conv = nn.Conv2d(channels_in, channels_out, kernel_size, padding=padding)
        self.sigmoid = nn.Sigmoid()

    def forward(self, gate, output):
        gate = self.conv(gate)
        gate = self.sigmoid(gate)
        return output * gate

class AOGLU(nn.Module):
    def __init__(self, channels, kernel_size=1, padding=0):
        super(AOGLU, self).__init__()
        self.conv = nn.Conv2d(channels, channels, kernel_size, padding=padding)
        self.sigmoid = nn.Sigmoid()

    def forward(self, output):
        gate = self.conv(output)
        gate = self.sigmoid(gate)
        return output * gate

In [24]:
class Net(pl.LightningModule):
    def __init__(self, lr=3e-4,
                 act=[nn.ReLU(), nn.ReLU(), nn.ReLU()]):
        super(Net, self).__init__()
        self.lr = lr

        self.conv1 = nn.Conv2d(3, 16, 5)
        self.bn1 = nn.BatchNorm2d(16)
        self.act1 = act[0]
        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv2 = nn.Conv2d(16, 32, 5)
        self.bn2 = nn.BatchNorm2d(32)
        self.act2 = act[1]
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv3 = nn.Conv2d(32, 32, 3)
        self.bn3 = nn.BatchNorm2d(32)
        self.act3 = act[2]
        self.pool3 = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(32, 10)
        self.d1 = nn.Dropout(0.2)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        if isinstance(self.act1, PGLU) or isinstance(self.act1, APGLU):
            out = self.act1(x, out)
        else:
            out = self.act1(out)
        out2 = self.pool1(out)

        out = self.conv2(out2)
        out = self.bn2(out)
        if isinstance(self.act2, PGLU) or isinstance(self.act2, APGLU):
            out = self.act2(out2, out)
        else:
            out = self.act2(out)
        out3 = self.pool2(out)

        out = self.conv3(out3)
        out = self.bn3(out)
        if isinstance(self.act3, PGLU) or isinstance(self.act3, APGLU):
            out = self.act3(out3, out)
        else:
            out = self.act3(out)
        out = self.pool3(out)

        out = out.view(-1, 32)
        out = self.fc1(out)
        out = self.d1(out)
        return out

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.lr)

    def train_dataloader(self):
        return DataLoader(CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor()), batch_size=25000, shuffle=True)

    def val_dataloader(self):
        return DataLoader(CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor()), batch_size=10000, shuffle=False)

    def test_dataloader(self):
        return DataLoader(CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor()), batch_size=10000, shuffle=False)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log('val_loss', loss, prog_bar=True)
        accuracy = (y_hat.argmax(dim=1) == y).float().mean()
        self.log('val_acc', accuracy, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        return loss

In [27]:
seed_everything(42)
net = Net(act=[AOGLU(16), AOGLU(32), AOGLU(32)])
trainer = pl.Trainer(accelerator='gpu', max_epochs=256, accumulate_grad_batches=2)
trainer.fit(net)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name  | Type        | Params
---------------------------------------
0  | conv1 | Conv2d      | 1.2 K 
1  | bn1   | BatchNorm2d | 32    
2  | act1  | AOGLU       | 272   
3  | pool1 | MaxPool2d   | 0     
4  | conv2 | Conv2d      | 12.8 K
5  | bn2   | BatchNorm2d | 64    
6  | act2  | AOGLU       | 1.1 K 
7  | pool2 | MaxPool2d   | 0     
8  | conv3 | Conv2d      | 9.2 K 
9  | bn3   | BatchNorm2d | 64    
10 | act3  | AOGLU      

Sanity Checking: 0it [00:00, ?it/s]

Files already downloaded and verified


E:\Blake Projects\Transformer-Experiments\venv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Files already downloaded and verified


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=256` reached.


In [28]:
max_accuracy = trainer.callback_metrics['val_acc'].max()
print(max_accuracy)

tensor(0.5882)
